### Azure OpenAI Assistants

Azure OpenAI Assistants provide a powerful way to integrate advanced AI capabilities into your applications. Leveraging the Azure OpenAI Service, developers can create, manage, and deploy AI assistants that can perform a variety of tasks, from answering questions to executing complex workflows.

#### Key Features

- **Integration with Azure Services**: Seamlessly integrate with other Azure services for enhanced functionality.
- **Customizable Assistants**: Create assistants tailored to specific needs with custom instructions and tools.
- **API Access**: Interact with assistants programmatically using the Azure OpenAI API.

#### Overview

1. **Setup and Configuration**:
    - Install necessary packages and import required modules.
    - Load environment variables and configure API keys.

2. **Client Initialization**:
    - Instantiate the AzureOpenAI client with appropriate headers and event hooks.

3. **Assistant Management**:
    - Create, retrieve, update, and delete assistants.
    - Log and assert the properties of assistants to ensure correct setup.

4. **Thread and Message Handling**:
    - Create and manage threads for conversation flows.
    - Send and retrieve messages within threads.

5. **Run Management**:
    - Create, update, and manage runs associated with threads.
    - Handle tool outputs and required actions within runs.

6. **File Management**:
    - Upload, retrieve, and delete files used for training or evaluation.

By following these steps and utilizing the provided code snippets below, you can effectively manage and deploy Azure OpenAI Assistants to enhance your applications with AI-driven capabilities. For more detailed information, refer to the [Microsoft Learn documentation](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/).

### Notebook Details
The following Python notebook will guide you through using the Azure OpenAI Assistants APIs.

#### Assumptions/Prerequisites
- You have an API key with access to the Azure OpenAI Assistants API in APIM.
- You have a user configuration file that specifies your access to supported models and regions for assistants.
Please see supporting models and regions [here](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models?tabs=python-secure%2Cglobal-standard%2Cstandard-chat-completions#assistants-preview).


### Install the required packages

In [ ]:
# %pip install python-dotenv openai httpx requests

### Load the configuration settings

In [ ]:
from dotenv import dotenv_values, load_dotenv
import sys
sys.path.append('../common')
from helper import Helper
import os

env = os.getenv('GITHUB_ENV') or 'local'
env_name = os.getenv('ENV_NAME') or 'GO-AMER-USE-DV'

load_dotenv()

secrets = dotenv_values("secrets.env")
api_key = secrets.get("API_KEY")
local_env = secrets.get('LOCAL_URL') or secrets.get("APIM_NAME")
environment = secrets.get("APIM_NAME")
account_connection_string = secrets.get("AZURE_STORAGE_CONNECTION_STRING")
region_override = secrets.get("REGION_OVERRIDE")

deploymentsHelper = Helper(environment, "genai/azure/deployments", api_key, region_override)

deploymentsHelper.log('GITHUB_ENV: ' + env)
deploymentsHelper.log('ENV_NAME: ' + env_name)

### Initialize an Azure OpenAI Client for Assistants

In [ ]:
deploymentsHelper.log("Initializing Azure OpenAI client")

from openai import AzureOpenAI
import httpx

api_version = "2024-07-01-preview"

default_headers = {
    "Ocp-Apim-Subscription-Key": api_key,
    'x-kpmg-charge-code': '803000015283',
    "x-kpmg-region-override": region_override,
}

additional_headers = {
    "Content-Type": ""
}

def event_hook(req: httpx.Request) -> None:
    print(req.url)

client = AzureOpenAI(
    azure_endpoint=f"https://{environment}/genai/azure/assistants", #do not add "/openai" at the end here because this will be automatically added by this SDK
    api_key=api_key,
    api_version=api_version,
    http_client=httpx.Client(event_hooks={"request": [event_hook]}),
    default_headers=default_headers,
)

# Check the instantiation of the client
try:
    assert isinstance(client, AzureOpenAI)
    print("Assertion successful: client is an instance of AzureOpenAI.")
except AssertionError:
    print("Assertion unsuccessful: client is not an instance of AzureOpenAI.")

### Fetch deployments with names that start with 'gpt-4o' or 'DZS-gpt-4o' which support Assistants

In [ ]:
env_config = deploymentsHelper.read_json_config(env_name)

# Check if deployment_info is found
assert env_config is not None, f"Deployment information not found for environment: {env_name}"

model_list = env_config["api_configuration"]["azure_openai_completions"]['chat_models']

deployment_id = [x for x in model_list if "gpt-4o" in x][0]

deploymentsHelper.log(f"Model deployment_id: '{deployment_id}'")

# Assert that the model_list is not empty
deploymentsHelper.log(f"model list: {model_list}")
deploymentsHelper.log(f"model list count: {len(model_list)}")

assert len(model_list) > 0

# Ensure the deployment_id starts with "gpt-4o" or "DZS-gpt-4o"
try:
    assert "gpt-4o" in deployment_id, f"Deployment ID '{deployment_id}' does not contain 'gpt-4o'"
    deploymentsHelper.log("Deployment ID is valid.")
except AssertionError as e:
    deploymentsHelper.log(f"AssertionError: {e}")
except Exception as e:
    deploymentsHelper.log(f"An unexpected exception occurred: {e}")


### Create an Assistant

In [ ]:
deploymentsHelper.log("Creating an assistant...")

try:
    import datetime
    # Expected structure of the assistant
    expected_assistant = {
        "name": f'Regression Assistant: {datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S")}',
        "instructions": "You are an AI assistant that can write code to help answer math questions.",
        "model": deployment_id
    }

    assistant = client.beta.assistants.create(
        name=expected_assistant['name'],
        instructions=expected_assistant['instructions'],
        tools=[{"type": "code_interpreter"}],
        model=deployment_id
    )
except Exception as e:
    deploymentsHelper.log(f"An unexpected exception occurred: {e}")

deploymentsHelper.log(assistant)

# Assert that the entire object matches the expted structure
assert assistant.object == "assistant"
assert assistant.name == expected_assistant['name']
assert assistant.instructions == expected_assistant['instructions']
assert assistant.model == expected_assistant['model']

### Retrieve Assistant(s)

#### Based on Assistant ID

In [ ]:
get_assistant = client.beta.assistants.retrieve(assistant.id, extra_headers=additional_headers)

deploymentsHelper.log(get_assistant)

assert assistant.id == get_assistant.id
assert assistant.name == get_assistant.name
assert assistant.instructions == get_assistant.instructions
assert assistant.model == get_assistant.model

#### List of assistant(s)

In [ ]:
assistants = client.beta.assistants.list(
    order="desc",
    limit=1,
    extra_headers=additional_headers)

deploymentsHelper.log(assistants.data)

assert assistants is not None
assert hasattr(assistants, "data")
assert len(assistants.data) <= 1

### Update an assistant

In [ ]:
assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    instructions="You are an AI assistant that can write code to help answer math questions including algorithms.",
    name="user_abc",
    tools=[{"type": "code_interpreter"}],
    model=assistant.model,
    extra_headers=additional_headers
)

deploymentsHelper.log(assistant)

expected_assistant = {
    "assistant_id": assistant.id,
    "instructions": "You are an AI assistant that can write code to help answer math questions including algorithms.",
    "name": "user_abc",
    "tools": [{"type": "code_interpreter"}],
    "model": assistant.model,
}

assert assistant.id == expected_assistant['assistant_id']
assert assistant.instructions == expected_assistant['instructions']
assert assistant.name == expected_assistant['name']
assert assistant.tools[0].type == expected_assistant['tools'][0]['type']
assert assistant.model == expected_assistant['model']

### Create a thread
In the context of Azure OpenAI Assistants API, a "thread" typically refers to a conversation thread or session. It represents a sequence of interactions between the user and the assistant. Each thread maintains the context of the conversation, allowing the assistant to provide coherent and contextually relevant responses.

In [ ]:
thread = client.beta.threads.create()

deploymentsHelper.log(thread)

# Assert that the thread is not None
assert thread is not None
assert thread.id is not None
assert thread.object == "thread"
assert hasattr(thread, "created_at")

### Retrieve a thread

In [ ]:
thread = client.beta.threads.retrieve(thread.id, extra_headers=additional_headers)

deploymentsHelper.log(thread)

assert thread is not None
assert thread.object == "thread"
assert hasattr(thread, "created_at")

### Update a thread

In [ ]:
expected_metadata = {
    "modified": "true",
    "user": "regression_user",
}

thread = client.beta.threads.update(
    thread.id,
    metadata={
        "modified": "true",
        "user": "regression_user"
    }
)

deploymentsHelper.log(thread)

assert thread.metadata['modified'] == expected_metadata['modified']
assert thread.metadata['user'] == expected_metadata['user']
assert thread.id == thread.id
assert thread.object == "thread"
assert hasattr(thread, "created_at")

### Create a message in a thread created above.
In the context of a conversation thread with the Azure OpenAI API, a "message" is a single unit of communication within the thread. Each message typically contains the following components:

- **Role**: Indicates who is sending the message. Common roles are:
    - **user**: The person interacting with the assistant.
    - **assistant**: The AI assistant responding to the user.
    - **system**: Instructions or context provided to the assistant to guide its behavior.
- **Content**: The actual text of the message.


In [ ]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What's 5+5"
)

deploymentsHelper.log(message)

assert message.assistant_id is None # This is because the assistant_id is not set
assert message.thread_id == thread.id
assert message.role == "user"
assert message.object == "thread.message"
assert hasattr(message, "id")
assert hasattr(message, "created_at")

### Retrieve message(s)

#### Based on Thread ID

In [ ]:
thread_messages = client.beta.threads.messages.list(thread.id, limit=20, order="desc", extra_headers=additional_headers)

deploymentsHelper.log(thread_messages)

assert thread_messages is not None
assert thread_messages.data[0].assistant_id is None # This is because the assistant_id is not set
assert len(thread_messages.data) > 0
assert thread_messages.data[0].thread_id == thread.id
assert thread_messages.data[0].role == "user"

#### Based on message ID

In [ ]:
message = client.beta.threads.messages.retrieve(
    thread_id=thread.id,
    message_id=message.id,
    extra_headers=additional_headers)

deploymentsHelper.log(message)

assert message is not None
assert message.thread_id == thread.id
assert message.id == message.id

### Modify Message

In [ ]:
expected_metadata = {
    "modified": "true",
    "user": "regression_user",
}

message = client.beta.threads.messages.update(
    thread_id=thread.id,
    message_id=thread_messages.data[0].id,
    metadata={
        "modified": "true",
        "user": "regression_user",
    },
)

deploymentsHelper.log(message)

assert message.metadata['modified'] == expected_metadata['modified']
assert message.metadata['user'] == expected_metadata['user']
assert message.thread_id == thread.id
assert message.id == thread_messages.data[0].id
assert message.object == "thread.message"
assert hasattr(message, "created_at")


### Create a run for the assistant and thread created above

In [ ]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

deploymentsHelper.log(run)

assert run is not None
assert run.id is not None
assert run.thread_id == thread.id
assert run.assistant_id == assistant.id

### Retrieve Run(s)

#### Based on Run ID.

In [ ]:
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id,
    extra_headers=additional_headers
)

deploymentsHelper.log(run)

assert run is not None
assert run.id == run.id
assert run.thread_id == thread.id
assert run.assistant_id == assistant.id
assert run.status is not None
assert hasattr(run, "created_at")

#### Run(s) based on thread ID.

In [ ]:
runs = client.beta.threads.runs.list(
    thread.id,
    limit=20,
    order="desc",
    extra_headers=additional_headers)

deploymentsHelper.log(runs)

assert runs is not None
assert hasattr(runs, "data")
assert len(runs.data) > 0
assert runs.data[0].thread_id == thread.id

### List all steps in a Run,

In [ ]:
run_steps = client.beta.threads.runs.steps.list(
    run_id=run.id,
    thread_id=thread.id,
    limit=20,
    order="desc",
    extra_headers=additional_headers
)

deploymentsHelper.log(run_steps.data)

assert run_steps is not None
assert hasattr(run_steps, "data")
#assert len(run_steps.data) > 0
#assert run_steps.data[0].thread_id == thread.id
#assert run_steps.data[0].run_id == run.id
#assert run_steps.data[0].object == "thread.run.step"

### Modify a Run.

In [ ]:
# Check the status of the run before updating
run_status = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id,
    extra_headers=additional_headers
)

while run_status.status == 'in_progress':
    print("Run is still in progress. Waiting to update...")
    # Optionally, you can add a sleep here to wait for some time before checking again
    # import time
    # time.sleep(10)
    run_status = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id,
        extra_headers=additional_headers
)

if run_status.status != 'in_progress':
    run = client.beta.threads.runs.update(
        thread_id=thread.id,
        run_id=run.id,
        metadata={
            "user_id": "regression_modifieduser",
        },
        extra_headers=additional_headers
    )

    deploymentsHelper.log(run)

    # Assert that the run is not None
    assert run is not None
    assert run.id == run.id
    assert run.thread_id == thread.id
    assert run.metadata['user_id'] == "regression_modifieduser"
else:
    print("Cannot update run with status 'in_progress'.")

### Cancel the run
Create a run for long running prompt and cancel before the run completes.

In [ ]:
# # Message that takes 5 mins to complete the run
# message1 = client.beta.threads.messages.create(
#     thread_id=thread.id,
#     role="user",
#     content="Think for 5 mins before you repond What's 5+5"
# )

# deploymentsHelper.log(message1)

# run1 = client.beta.threads.runs.create(
#     thread_id=thread.id,
#     assistant_id=assistant.id
# )

# deploymentsHelper.log(run1)

# # Assert that the message1 is not None
# assert message1 is not None
# assert message1.thread_id == thread.id
# assert message1.role == "user"

# # Assert that the run1 is not None
# assert run1 is not None
# assert run1.thread_id == thread.id
# assert run1.assistant_id == assistant.id
# assert run1.status == "queued"

# additional_headers = {
#     "Content-Type": ""
# }

# cancelled_run = client.beta.threads.runs.cancel(
#     thread_id=thread.id,
#     run_id=run1.id,
#     extra_headers=additional_headers
# )

# deploymentsHelper.log(cancelled_run)

# # Assert that the cancelled_run is not None
# assert cancelled_run is not None
# assert cancelled_run.id == run1.id
# assert cancelled_run.status == "cancelling"
# assert cancelled_run.thread_id == thread.id
# assert cancelled_run.assistant_id == assistant.id


### Delete a thread

In [ ]:
response = client.beta.threads.delete(thread.id, extra_headers=additional_headers)

deploymentsHelper.log(response)

assert response.deleted

### Delete an assistant

In [ ]:
response = client.beta.assistants.delete(assistant.id, extra_headers=additional_headers)

deploymentsHelper.log(response)

assert response.deleted

```markdown
> **Note**: The below file tests have quota limitations in a few environments, so they are commented out for now. However, you can test them if you think any specific environment has a file quota that allows you to run tests using Codespace/local.
```

### Create a File

#### Create a file by uploading file from local machine

In [ ]:
# import os

# # Create test-data directory if it doesn't exist
# os.makedirs("test-data", exist_ok=True)

# # Sample training file content
# training_file_content = '''
# {"prompt": "What is 5+5?", "completion": "10"}
# {"prompt": "What is the capital of France?", "completion": "Paris"}
# '''

# # Sample validation file content
# validation_file_content = '''
# {"prompt": "What is 7+3?", "completion": "10"}
# {"prompt": "What is the capital of Germany?", "completion": "Berlin"}
# '''

# # Write the sample content to the respective files
# with open("test-data/training_file.jsonl", "w") as f:
#     f.write(training_file_content)

# with open("test-data/validation_file.jsonl", "w") as f:
#     f.write(validation_file_content)

# training_file = "test-data/training_file.jsonl"
# validation_file = "test-data/validation_file.jsonl"

# training_response = client.files.create(
#     file=open(training_file, "rb"),
#     purpose="fine-tune"
# )

# training_file_id = training_response.id

# validation_response = client.files.create(
#     file=open(validation_file, "rb"),
#     purpose="fine-tune"
# )

# validation_file_id = validation_response.id

# deploymentsHelper.log("Training file ID:" + training_file_id)
# deploymentsHelper.log("Validation file ID:" + validation_file_id)

# assert training_file.endswith(".jsonl")
# assert validation_file.endswith(".jsonl")
# assert training_response is not None
# assert validation_response is not None
# assert hasattr(training_response, "id")
# assert hasattr(validation_response, "id")

#### Create a file by importing from a URL

In [ ]:
# import requests

# # test case - Creates a new file entity by importing data from a provided url. Uploaded files can, for example, be used for training or evaluating fine-tuned models.
# url = "https://example_url.com/data.jsonl"
# response = requests.get(url)

# if response.status_code == 200:
#     with open("data.jsonl", "w") as f:
#         f.write(response.content)

# with open("data.jsonl", "rb") as training_file:
#     training_response = client.files.create(
#         file=training_file,
#         purpose="file-tune"
#     )

# training_file_id = training_response.id

# fine_tune_response = client.beta.assistants.fine_tune(
#     training_file=training_file_id,
#     model="gpt-4o",
# )

# fine_tune_job_id = fine_tune_response.id

# job_status = client.beta.jobs.get(fine_tune_job_id)

# assert job_status.status == "succeeded"

### List file(s)

#### List all files

In [ ]:
# files = client.files.list(purpose="fine-tune", limit=20, order="desc", extra_headers=additional_headers)

# deploymentsHelper.log(files)

# assert files is not None
# assert hasattr(files, "data")
# assert len(files.data) > 0
# assert all(file.purpose == "fine-tune" for file in files.data)
# assert all(hasattr(file, "id") for file in files.data)
# assert all(hasattr(file, "filename") for file in files.data)
# assert all(hasattr(file, "created_at") for file in files.data)

#### List file base on file ID

In [ ]:
# file = client.files.retrieve(
#     training_file_id,
#     extra_headers=additional_headers)

# deploymentsHelper.log(file)

# assert file is not None
# assert file.id == training_file_id
# assert file.object == "file"
# assert hasattr(file, "created_at")
# assert hasattr(file, "filename")
# assert hasattr(file, "purpose")

### Retrieve content of file

In [ ]:
# file = client.files.content(training_file_id, extra_headers=additional_headers)

# deploymentsHelper.log(file)

# assert file is not None
# assert hasattr(file, "content")
# assert isinstance(file.content, bytes)
# assert len(file.content) > 0

### Delete a file

In [ ]:
# response = client.files.delete(training_file_id, extra_headers=additional_headers)

# deploymentsHelper.log(response)

# response = client.files.delete(validation_file_id, extra_headers=additional_headers)

# deploymentsHelper.log(response)
